В домашнем задании нужно решить задачу классификации точек наиболее эффективно. Для этого в работе необходимо применить различные методы по отбору признаков. Отбор признаков предпочтительнее осуществлять основываясь на математическом аппарате, поэтому данные для этого задания будут сгенерированы, чтобы избежать признаков с физическим смыслом.

In [ ]:
import numpy as np
import pandas as pd 
import scipy.stats 
import seaborn as sns
import matplotlib.pyplot as plt  

1. Сгенерируйте данные.

In [ ]:
from sklearn.datasets import make_classification

x_data_generated, y_data_generated = make_classification(scale=1)

In [ ]:
x_data_generated

In [ ]:
y_data_generated

2. Постройте модель логистической регрессии и оцените среднюю точность.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_data_generated, y_data_generated, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

model.score(X_test, y_test) 

3. Используйте статистические методы для отбора признаков:
* Выберите признаки на основе матрицы корреляции.
* Отсеките низковариативные признаки (VarianceThreshold).
* Повторите п. 2 на отобранных признаках в п. 3a, п. 3b.

In [ ]:
data = pd.DataFrame(x_data_generated, columns=[f'feature_{i}' for i in range(x_data_generated.shape[1])])
data['target'] = y_data_generated
data.head()

In [ ]:
corr_matrix = data.corr()

In [ ]:
plt.figure(figsize=(20, 15))
sns.heatmap(corr_matrix, annot=True, fmt=".2f")
plt.title('Корреляционная матрица признаков')
plt.show()

In [ ]:
# Выберем признаки на основе матрицы корреляции
data_corr = data.corr()['target'].drop('target')
high_corr_target = data_corr[data_corr.abs() > 0.5]
high_corr_target

In [ ]:
# Выводим список признаков
high_corr_features = high_corr_target.index.tolist()
high_corr_features

In [ ]:
features_corr = len(high_corr_features)
features_corr

In [ ]:
X = data[high_corr_features]
y = data['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

score_corr = model.score(X_test, y_test) 
score_corr

In [ ]:
# Оценим дисперсию каждого признака
dispersions = np.var(x_data_generated, axis=0)
for i, var in enumerate(dispersions):
    print(f'{i}: дисперсия = {var:.3f}')

In [ ]:
# Уберем низковариативные признаки с помощью VarianceThreshold (порог не ниже 0.9)
from sklearn.feature_selection import VarianceThreshold

# Удаляем целевую переменную из данных
data_ = data.drop(columns='target')

feature_names = data_.columns
selector = VarianceThreshold(threshold=0.9)
x_data_reduced = selector.fit_transform(data_)
mask = selector.get_support()

# Выводим список признаков
selected_feature_names = feature_names[mask]
selected_feature_names

In [ ]:
features_variance = len(selected_feature_names)
features_variance

In [ ]:
X = data[selected_feature_names]
y = data['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

score_variance = model.score(X_test, y_test) 
score_variance

4. Осуществите отбор признаков на основе дисперсионного анализа:
* Выберите 5 лучших признаков с помощью скоринговой функции для классификации f_classif (SelectKBest(f_classif, k=5)).
* Повторите п. 2 на отобранных признаках.

In [ ]:
# Выбор 5 лучших признаков с помощью f_classif 
from sklearn.feature_selection import SelectKBest, f_classif

feature_names = data.drop(columns='target').columns
selector = SelectKBest(f_classif, k=5)
x_data_reduced = selector.fit_transform(data.drop(columns='target'), data['target'])
mask = selector.get_support()

# Cписок оставшихся признаков
selected_feature_names = feature_names[mask]
selected_feature_names

In [ ]:
# Количество признаков по отбору
features_classif = len(selected_feature_names)
features_classif

In [ ]:
X = data[selected_feature_names]
y = data['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

score_classif = model.score(X_test, y_test) 
score_classif

5. Отбор с использованием моделей:
* Реализуйте отбор признаков с помощью логистической регрессии. Отобранные признаки подайте далее на вход в 
* Реализуйте отбор признаков с помощью модели RandomForest и встроенного атрибута feature_impotance.
* Повторите п. 2 на отобранных признаках в п. 5a, п. 5b.

In [ ]:
# Делим данные на train, test
X_train, X_test, y_train, y_test = train_test_split(x_data_generated, y_data_generated, test_size=0.2, random_state=42)

In [ ]:
# Отбор признаков с помощью логистической регрессии
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# Коэффициенты
coef_abs = np.abs(model.coef_[0])

# 5 лучших
k = 5
top_indices = np.argsort(coef_abs)[-k:]

X_train_lr = X_train[:, top_indices]
X_test_lr = X_test[:, top_indices]

selected_feature_names = feature_names[top_indices]
selected_feature_names

In [ ]:
features_lr = len(selected_feature_names)
features_lr

In [ ]:
X_train_data = pd.DataFrame(X_train_lr, columns=selected_feature_names)
X_test_data = pd.DataFrame(X_test_lr, columns=selected_feature_names)

model_selected = LogisticRegression(random_state=42)
model_selected.fit(X_train_data, y_train)

score_lr = model_selected.score(X_test_data, y_test)
score_lr

In [ ]:
# Отбор признаков с помощью RandomForest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

importances = rf.feature_importances_

k = 5
top_indices_rf = np.argsort(importances)[-k:]

X_train_rf = X_train[:, top_indices_rf]
X_test_rf = X_test[:, top_indices_rf]

selected_feature_names = feature_names[top_indices_rf]
selected_feature_names

In [ ]:
features_rf = len(selected_feature_names)
features_rf

In [ ]:
X_train_data = pd.DataFrame(X_train_rf, columns=selected_feature_names)
X_test_data = pd.DataFrame(X_test_rf, columns=selected_feature_names)

model_selected = RandomForestClassifier(random_state=42)
model_selected.fit(X_train_data, y_train)

score_rf = model_selected.score(X_test_data, y_test)
score_rf

6. Перебор признаков:
* SequentialFeatureSelector.
* Повторите п. 2 на отобранных признаках.

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

sfs = SequentialFeatureSelector(
    LogisticRegression(random_state=42),
    n_features_to_select=5,
    direction='forward'
)

sfs.fit(X_train, y_train)

selected_mask = sfs.get_support()
selected_indices = np.where(selected_mask)[0]
selected_feature_names = feature_names[selected_indices]
selected_feature_names

In [ ]:
features_selector = len(selected_feature_names)
features_selector

In [ ]:
X = data[selected_feature_names]
y = data['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

score_selector = model.score(X_test, y_test)
score_selector

7. Сформулируйте выводы по проделанной работе:
* Сделайте таблицу вида |способ выбора признаков|количество признаков|средняя точность модели|.

In [ ]:
data = {'Отбор признаков': 
        ['Матрица корреляции', 'VarianceThreshold', 'f_classif', 'LogisticRegression', 'RandomForest', 'SequentialFeatureSelector'],
        'Количество признаков': 
        [features_corr, features_variance, features_classif, features_lr, features_rf, features_selector],
        'Точность модели': 
        [score_corr, score_variance, score_classif, score_lr, score_rf, score_selector]}

In [ ]:
# В зависимости от сгенерированныъ данных результаты разнятся в зависимости от выбора метода отбора
df = pd.DataFrame(data)
df